In [9]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [27]:
path_to_db_local = "C:/Users/admin/Downloads/Practicum/Projects/8_telecom_analysis/telecomm_csi (1).db"
# путь к БД на платформе
path_to_db_platform = 'https://code.s3.yandex.net/datasets/telecomm_csi.db'
# итоговый путь к БД
path_to_db = None


if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [28]:
query = """
SELECT user_id,
       lt_day,
       CASE 
                 WHEN lt_day < 365 THEN 'new'
                 WHEN lt_day > 365 THEN 'old'
             END AS is_new,
       age,
       CASE 
                 WHEN gender_segment = '1.0' THEN 'женщина'
                 WHEN gender_segment = '0.0' THEN 'мужчина'
             END AS gender,
       os_name,
       cpe_type_name,
       loc.country,
       loc.city,
       SUBSTRING(a.title, 4) AS age_segment,
       SUBSTRING(t.title, 4) AS traffic_segment,
       SUBSTRING(l.title, 4) AS lifetime_segment,
       nps_score,
       CASE 
                 WHEN nps_score >= 9 THEN 'promoters'
                 WHEN nps_score = 7 OR nps_score = 8 THEN 'passives'
                 WHEN nps_score <= 6 THEN 'detractors'
             END AS nps_group
              
      FROM user AS u

LEFT OUTER JOIN location AS loc ON u.location_id = loc.location_id
LEFT OUTER JOIN age_segment AS a ON u.age_gr_id = a.age_gr_id
LEFT OUTER JOIN traffic_segment AS t ON u.tr_gr_id = t.tr_gr_id
LEFT OUTER JOIN lifetime_segment AS l ON u.lt_gr_id = l.lt_gr_id      


"""

In [29]:
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,is_new,age,gender,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,old,45.0,женщина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,promoters
1,A001WF,2344,old,53.0,мужчина,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,promoters
2,A003Q7,467,old,57.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,promoters


In [30]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

Tableau: <https://public.tableau.com/views/telecomm3_16790800105680/NPS_3?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link>

Презентация: <https://docs.google.com/presentation/d/11Cx76iwx-tGQcz4I3cQS3Mwveb5_VjC6T2_OudY143s/edit?usp=sharing>